# Explore Jaccard Similarity of Playlists

## Initialize Notebook

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import pandas as pd
import matplotlib as plt

import mpd

In [ ]:
# Will allow us to embed images in the notebook
%matplotlib inline
# change default plot size
plt.rcParams['figure.figsize'] = (15,10)

## Load Data

In [ ]:
mpd_all=mpd.load(spark, "onebig", 1)

## Vectorize the playlist tracks

Convert the playlists to sparse feature vectors using countVectors.  This prepares the data set for use in machine learning functions.

Use a standard mllib pipeline to fit the data to the CountVectorizer() and transform the dataframe into one that contains the sparse vector features.

This is based on initial countVector exploration form mpd-expore notebook.

In [ ]:
pDF=mpd_all.select("pid", "tracks.track_uri")

model, result = mpd.vectorizecol(pDF, "track_uri", "features")

In [ ]:
result.show(10)

In [ ]:
trackcv = model

In [ ]:
type(trackcv.vocabulary)

In [ ]:
trackcv.vocabulary[0]

In [ ]:
from pyspark.sql.types import StringType

tvdf = sqlContext.createDataFrame(trackcv.vocabulary, StringType())

In [ ]:
from pyspark.sql.functions import monotonically_increasing_id

tvdf = tvdf.withColumn("id", monotonically_increasing_id())

In [ ]:
tvdf.show(5, truncate=False)

In [ ]:
from pyspark.sql.functions import explode
tname=mpd_all.select(explode("tracks").alias("tracks")).select("tracks.track_name", "tracks.track_uri").distinct()

In [ ]:
tname.printSchema()

In [ ]:
tvdf.join(tname, tname.track_uri == tvdf.value).orderBy("id").show(5)

## Understand sparse vector type

The features column now contains a sparse vector.  We can see the first playlist in its sparse vector format.

Note that the type of returned data is a Row and it contains a field named 'features' which contains the sparse vector.

In [ ]:
result.select("features").head(1)

In [ ]:
result.select('features').show(1, truncate=False)

It's easier to understand desplay types by looking at constructed sparse vector.

The vector is constructed from a string representation of a sparse vector which is a list that contains three elements: vector length, values, and value indecies.

In [ ]:
from pyspark.mllib.linalg import Vectors

Vectors.parse('(3, [0], [2])')

Note the above conversion of the string representation of a sparse vector to a sparse vector datatype shows how these two formats are displayed.

When using the show() method the string representation is returned.  When using the head on the feature we actually see a native sparse vector datatype in a row.  This shows countvectorizer is actually producing sparse vector types, it's only show() that turns it into a string.

Since the feture is actually a row with a sparse vector in it, may be able take the feature column as a sparse matrix already and perform the distance measures on it.

## Compute minhash LSH

First attempt at producing a distance metric for the playlists to understand their similarity.

In [ ]:
from pyspark.ml.feature import MinHashLSH
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col

In [ ]:
mh = MinHashLSH(inputCol="features", outputCol="hashes", numHashTables=5)

The result dataframe still contains all playlists

In [ ]:
result.count()

### Apply minhash to limited subset to avoid error

Trying to apply the minhash to the entire data set raises an error that i don't understand.  Using only 10 elements avoids it but doesn't give much meaningful information when computing the Jaccard similarity.

In [ ]:
model = mh.fit(result.limit(10))

In [ ]:
transformA = model.transform(result.limit(10))

In [ ]:
transformB = model.transform(result.limit(10))

In [ ]:
transformA.select("hashes").limit(1)

In [ ]:
model.approxSimilarityJoin(transformA, transformB, 0.6, distCol="JaccardDistance")

Apply do an approximate simliarlty join on the truncated data

In [ ]:
jd=model.approxSimilarityJoin(transformA, transformB,10.0, distCol="JaccardDistance")\
    .select(col("datasetA.pid").alias("idA"),
            col("datasetB.pid").alias("idB"),
            col("JaccardDistance"))

In [ ]:
jd.show()

Attempt a sampled subset of the data.

In [ ]:
train = result.rdd.sample(False, .01, 1)
train = train.toDF()

In [ ]:
train.show()

model.approxNearestNeighbors(dfA, key, 2).show()

### Inspect the minhash 

In [ ]:
from pyspark.sql.functions import col, expr, when

In [ ]:
result.show(1)

In [ ]:
transformA.select("hashes").show(5, truncate=False)

## Figure out how to work with sparse vectors

Understand how to apply methods to colums via RDD converstion and the map() function.

In [ ]:
result.dtypes

Able to access the column as a sparse vector type but this doesn't let me operate on it

In [ ]:
result.select("features").rdd.map(lambda x: type(x.features)).take(5)

Able to call the sparseVector methods on the x element.

To return this as a dataframe on which we can run summary stats, the key is to [cast the lambda result as a Row](https://spark.apache.org/docs/latest/sql-programming-guide.html#inferring-the-schema-using-reflection). The toDF() works on rows.

In [ ]:
from pyspark.sql import Row


sparselen=result.select("features").rdd.map(lambda x: Row(length=x.features.numNonzeros())).toDF()

In [ ]:
sparselen.printSchema()

The summary stats are promising and show we are close to the same averages as with the full play lists data set.

In [ ]:
sparselen.describe().show()

Add an index to the RDD to support joins with the original results.  The goal is to select only the sparse vectors that are not all zero valued.

In [ ]:
slen=result.select("features").rdd.map(lambda x:x.features.numNonzeros()).zipWithIndex().toDF()

In [ ]:
slen.show(10)

### Compare the extracted features to the original feature set

Use the summary stats on the original data to confirm that we haven't lost much detail from the default vocabulary size.

In [ ]:
origlen = result.select("track_uri").rdd.map(lambda x: Row(length=len(x[0]))).toDF()

In [ ]:
origlen.describe().show()

Need to remove the zero length vectors to run the LSH hash algorithms.

How many zero vectors do we have?

In [ ]:
sparselen.where(sparselen.length == 0).count()

In [ ]:
origlen.where(origlen.length == 5).count()

Only about half of the playlists have disappeared completely from the minimum length in the original dataset.

Need to get the vector length into the original dataframe so I can select by size when i run the algorithms

## Add the length data to the result dataframe

Understand the [approaches to add columns to dataframes](https://stackoverflow.com/a/33683462)and use the user defined function, udf, to map the sparse vector method to the column.`

Columns can oly be added using the fullowing approaches:
- literal value
- transforming an existing column
- using join
- using funtion/udf

Adding an arbitrary RDD result is basically a join by adding an index to the RDD output.
- add row num to existing dataframe (i have pid)
- zipwithindex on RDD
- join

In [ ]:
result.printSchema()

Compute the lenth of the playlist array using the built-in size() function.

In [ ]:
from pyspark.sql.functions import size

r2=result
r2=r2.withColumn("plen", size(r2.track_uri))

Build a udf to compute the sparse vector non zero entry counts.  This is akin the the map() method for an RDD.

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

vectorlength = udf(lambda x: x.numNonzeros(), IntegerType())

In [ ]:
r2=r2.withColumn("vlen", vectorlength(r2.features))

In [ ]:
r2.printSchema()

Now have a dataframe that has been extended with length information.  This can be used in selects to elliminate data that throws errors on minhash (sparse vectors with all zero entries).

In [ ]:
r2.show(10)

## Compute Jaccard similarity across full data set

Use only the sparse vectors that don't have all entries as zero because [MinHash can't transform empty sets](https://spark.apache.org/docs/2.2.0/ml-features.html#minhash-for-jaccard-distance).

In [ ]:
sparsevec = r2.where(r2.vlen > 0)
sparsevec.count()

Sample the data set to get a more managable data size. 

Shouldn't need to do this since the 

In [ ]:
sparsevec= sparsevec.rdd.sample(False, .01, 1).toDF()

In [ ]:
sparsevec.count()

In [ ]:
model = mh.fit(sparsevec)

In [ ]:
transformA = model.transform(sparsevec)

In [ ]:
transformB = model.transform(sparsevec)

In [ ]:
jd=model.approxSimilarityJoin(transformA, transformB,0.6, distCol="JaccardDistance")\
    .select(col("datasetA.pid").alias("idA"),
            col("datasetB.pid").alias("idB"),
            col("JaccardDistance"))

In [ ]:
jd.where(jd.JaccardDistance > 0.0).show(10)

In [ ]:
jd.show(10)

In [ ]:
jd.count()

In [ ]:
jd2=model.approxSimilarityJoin(transformA, transformB,0.6, distCol="JaccardDistance")

In [ ]:
jd2.printSchema()

In [ ]:
type(result.features.getItem(0))

Extract a single sparse vector to feed to the nearest neighbor search. Pipe it through rdd to get a list from which to get one element.

In [ ]:
testpl=result.select("features").rdd.map(lambda x: x.features).take(1)[0]

In [ ]:
type(testpl)

In [ ]:
print("Approximately searching dfA for 2 nearest neighbors of the key:")
model.approxNearestNeighbors(transformA, testpl, 2).show()

Inspecting the search playlist and the two nearest neighbors shows that the two returned playlists are also named for the "throwback" theme.  Clearly there is a pattern detected here.

The songs not in the search playlist but in the nearest playlist would make good recommendations.

In [ ]:
mpd_all.where(mpd_all.pid==0).show()

In [ ]:
mpd_all.where(mpd_all.pid==131786).show()

In [ ]:
mpd_all.where(mpd_all.pid==262260).show()

See how the 10 nearest neighbors stack up.

In [ ]:
first10nn = model.approxNearestNeighbors(transformA, testpl, 10)

They are all at a similar distance.  Would be helpful to understand the distribution of distances.  Feel like i need some sort of graphing viz for that. 

In [ ]:
first10nn.printSchema()

In [ ]:
mpd.plothist(first10nn, "distCol", 11)

### Track rank for recommended lists

In [ ]:
f10tracks=first10nn.select(explode("track_uri").alias("track_uri"))

In [ ]:
f10tracks.printSchema()

In [ ]:
f10tracks.show()

In [ ]:
trackrank = f10tracks.select("track_uri").groupby("track_uri").count().sort(f.col("count").desc())

In [ ]:
Y=trackrank.select("count").toPandas()

In [ ]:
Y.size

In [ ]:
X=pd.DataFrame({'X': range(1,Y.size+1,1)})

In [ ]:
plt.pyplot.scatter(X,Y)

In [ ]:
grank=trackrank.join(tvdf, trackrank.track_uri == tvdf.value)

In [ ]:
grank.orderBy(desc("count"), desc("id")).show()

In [ ]:
grank.count()

### Gather info from global data set

Use join to get global data on recommended lists like the playlist name.

In [ ]:
first10nn.schema

In [ ]:
first10nn = first10nn.withColumnRenamed("pid", "recpid")

In [ ]:
def pnamelookup(pid):
    mpd_all.where(mpd_all.pid==pid)
    
pnamelookup=udf(pnamelookup)

In [ ]:
pln=first10nn.join(mpd_all, mpd_all.pid == first10nn.recpid)

In [ ]:
pln.printSchema()

In [ ]:
pln.select("name", "modified_at", "num_edits", "num_followers","plen").show()

In [ ]:
from pyspark.sql.functions import explode

In [ ]:
pDF=pln.select("pid", explode("tracks").alias("track")).select("track.*")

In [ ]:
pDF.printSchema()

In [ ]:
pDF.select("track_name").show()

In [ ]:
pDF.count()

In [ ]:
recflat=mpd.playlist_flatten(pln.select("pid", "name", "modified_at", "num_edits", "num_followers","plen", "tracks"))

In [ ]:
recflat.show(10)

In [ ]:
for name in pln.select("name"):
    print(name)

## Explore challenge set

In [ ]:
mpd_test=spark.read.json("../mpd-challenge/challenge_set.json", multiLine=True)

In [ ]:
mpd_test.printSchema()

In [ ]:
mpd_test.show()

In [ ]:
cpl=mpd_test.select(explode("playlists").alias("playlist"))

In [ ]:
cpl.printSchema()

In [ ]:
type(cpl)

In [ ]:
cpl.schema

In [ ]:
cpl.show(3, truncate=False)

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Convenience function for turning JSON strings into DataFrames.
def jsonToDataFrame(json, schema=None):
  # SparkSessions are available with Spark 2.0+
  reader = spark.read
  if schema:
    reader.schema(schema)
  return reader.json(sc.parallelize([json]))


In [ ]:
events = jsonToDataFrame("""
{
  "a": [1, 2]
}
""")

display(events.select(explode("a").alias("x")))



In [ ]:
events.select(explode("a").alias("x")).show()

In [ ]:
events = jsonToDataFrame("""
{
  "a": [{"b": 6, "c": 5}, {"b": 6, "c": 5}]
}
""")

display(events.select(explode("a").alias("x")))

In [ ]:
te=events.select(explode("a").alias("x"))

In [ ]:
te.printSchema()

In [ ]:
te.select("x.c").show()

In [ ]:
te.select("x.b", "x.c").show()

In [ ]:
recdf=cpl.select("playlist.name", "playlist.num_holdouts", "playlist.pid", "playlist.num_tracks", "playlist.tracks", "playlist.num_samples")

In [ ]:
recdf.show(5)

In [ ]:
recdf.describe("num_samples").show()

In [ ]:
# Doing the heavy lifting in Spark. We could leverage the `histogram` function from the RDD api

gre_histogram = recdf.select("num_samples").rdd.flatMap(lambda x: x).histogram(11)

# Loading the Computed Histogram into a Pandas Dataframe for plotting
pd.DataFrame(
    list(zip(*gre_histogram)), 
    columns=['bin', 'frequency']
).set_index(
    'bin'
).plot(kind='bar');

In [ ]:
pln.printSchema()

In [ ]:
import importlib
importlib.reload(mpd)

In [ ]:
mpd.plothist(pln, "num_edits", 11)

In [ ]:
mpd.plothist(recdf, "num_samples", 27)

In [ ]:
mpd.plothist(recdf, "num_tracks", 27)

In [ ]:
mpd.plothist(pln, "num_tracks", 27)

In [ ]:
mpd.plothist(mpd_all, "num_tracks", 27)

In [ ]:
mpd.plothist(mpd_all, "num_followers", 27)